# Реализация класса Similarity1

Класс вычисляет сходство между вектором контекста декодера и векторами контекста энкодера.  
Сходство определяется как скалярное произведение:  
$\text{sim}(h, s) = h^T s = \sum_i h_i \cdot s_i$


In [ ]:
import torch
import torch.nn as nn

class Similarity1(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, encoder_states: torch.Tensor, decoder_state: torch.Tensor):
        # encoder_states.shape = [T, N]
        # decoder_state.shape = [N]

        decoder_state = decoder_state.unsqueeze(0) # [N] -> [1, N]
        similarity = torch.sum(encoder_states * decoder_state, dim=1) # [T, N] * [1, N] -> [T]

        return similarity

# Реализация класса для вычисления сходства

Класс вычисляет сходство между вектором контекста декодера и векторами контекста энкодера.  
Сходство определяется как:

$\text{sim}(h, s) = \text{fc3}\left(\tanh\left(\text{fc1}(h) + \text{fc2}(s)\right)\right)$

где:
- $h$ — вектор состояния из энкодера.
- $s$ — вектор состояния из декодера.
- $\text{fc1}$, $\text{fc2}$, и $\text{fc3}$ — это линейные слои (fully connected layers), которые обучаются.


In [ ]:
import torch
import torch.nn as nn


class Similarity2(nn.Module):
    def __init__(self, encoder_dim: int, decoder_dim: int, intermediate_dim: int):
        super().__init__()

        self.fc1 = nn.Linear(encoder_dim, intermediate_dim)
        self.fc2 = nn.Linear(decoder_dim, intermediate_dim)
        self.fc3 = nn.Linear(intermediate_dim, 1)
        self.tanh = nn.Tanh()

    def forward(self, encoder_states: torch.Tensor, decoder_state: torch.Tensor):
        # encoder_states.shape = [T, N]
        # decoder_state.shape = [N]

        h = self.fc1(encoder_states)  # [T, intermediate_dim]
        s = self.fc2(decoder_state)  # [intermediate_dim]

        f_act = torch.tanh(h + s)

        sim = self.fc3(f_act)

        return sim